# Methods & Results

In [ ]:
library(tidyverse)
library(tidymodels)

In [ ]:
players_data <- read_csv("data/players.csv")
players_data

Our analysis starts by importing the dataset directly from the data folder.

In [ ]:
players_tidy <- players_data |>
  select (experience, subscribe, played_hours, Age) |>
  filter (Age != 17) |>
  mutate (experience = as_factor(experience),
        subscribe = as_factor(subscribe))
head(players_tidy)

After loading, we clean the dataset to include only the relevant variables: experience, subscribe, played_hours, and Age. These variables are chosen because they relate closely to user behavior and potential factors influencing subscription decisions. Players aged exactly 17 are removed, because they form an unrepresentative group that could introduce noise. Converting experience and subscribe to categorical types ensures that these variables are treated correctly during modeling, especially since subscribe is the outcome we aim to predict.

In [ ]:
p1 <- players_tidy |>
  ggplot(aes(x = played_hours, fill = subscribe)) +
  geom_histogram(binwidth = 5, alpha = 0.7, position = "identity") +
  labs(title = "Figure 1: Distribution of Played Hours by Subscription",
       x = "Played Hours", y = "Count") +
  theme_minimal()
p1

In [ ]:
p2 <- players_tidy |>
  ggplot(aes(x = experience, y = played_hours, fill = experience)) +
  geom_boxplot() +
  labs(title = "Figure 2: Played Hours by Experience Level",
       x = "Experience Level", y = "Played Hours") +
  theme_minimal()
p2

The next step involves visual exploration of the data. A histogram is created to compare how playtime is distributed across subscription statuses. This visualization provides an intuitive way to assess whether more engaged users tend to subscribe and whether there is a pattern that could inform predictions. The bin width is chosen to balance granularity and clarity. A second plot—a boxplot—illustrates how playtime differs across experience levels. This helps us explore whether experience is related to the amount of time spent playing, which might be another indicator of subscription behavior. Both visualizations serve to uncover possible relationships among the variables and the target, guiding feature selection and model expectations.

In [ ]:
set.seed(123)
data_split <- initial_split(players_tidy, prop = 0.8, strata = subscribe)
train_data <- training(data_split)
test_data <- testing(data_split)

The dataset is then split into training and testing sets with an 80/20 ratio. This ensures enough data is available for training while reserving a portion for final model evaluation. Stratifying by the outcome variable ensures that both subsets have similar class distributions, which is critical for classification tasks to avoid biased or misleading results.

In [ ]:
knn_recipe <- recipe(subscribe ~ played_hours + Age + experience, data = train_data) |>
  step_normalize(all_numeric_predictors()) |>
  step_dummy(all_nominal_predictors())

We then create a preprocessing recipe that outlines how the data should be transformed prior to modeling. Normalizing numeric variables is essential because the model we’re using is sensitive to scale—larger-scale features could dominate smaller ones and distort the distance calculations. Dummy encoding is applied to the experience variable, allowing the categorical levels to be represented in a way the model can interpret and process effectively. This preprocessing ensures the model receives clean, consistent input without hidden biases from scaling or encoding mismatches.